## Fashion-mnist

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()
#train_images and train_labels are independent and dependent feature

In [4]:
#scaling
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
len(train_images)

60000

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [24]:
def build_model(hp):
    model=keras.Sequential([keras.layers.Conv2D(filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                           activation='relu',input_shape=(28,28,1)
                                               ),
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                                kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                                activation='relu'
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                                               activation='relu'
                                              ),
                            keras.layers.Dense(10, activation='softmax')])
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

In [25]:
import keras_tuner
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [26]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [27]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 09m 08s]
val_accuracy: 0.8830000162124634

Best val_accuracy So Far: 0.906000018119812
Total elapsed time: 01h 16m 37s
INFO:tensorflow:Oracle triggered exit


In [29]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        128064    
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2048080   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,179,034
Trainable params: 2,179,034
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 357s 211ms/step - loss: 0.1688 - accuracy: 0.9366 - val_loss: 0.2594 - val_accuracy: 0.9087
Epoch 5/10
1688/1688 [==============================] - 385s 228ms/step - loss: 0.1306 - accuracy: 0.9503 - val_loss: 0.2792 - val_accuracy: 0.9112
Epoch 6/10
1688/1688 [==============================] - 366s 217ms/step - loss: 0.0985 - accuracy: 0.9632 - val_loss: 0.2950 - val_accuracy: 0.9110
Epoch 7/10
1688/1688 [==============================] - 325s 192ms/step - loss: 0.0753 - accuracy: 0.9716 - val_loss: 0.3190 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 316s 187ms/step - loss: 0.0586 - accuracy: 0.9785 - val_loss: 0.3885 - val_accuracy: 0.9117
Epoch 9/10
1688/1688 [==============================] - 309s 183ms/step - loss: 0.0471 - accuracy: 0.9823 - val_loss: 0.4403 - val_accuracy: 0.9090
Epoch 10/10
1688/1688 [==============================] - 371s 220ms/step - loss: 0.0381 - accuracy: 0.9858 - val